# 1

In [1]:
%1
NGS_Block = dlmread('Q1/RESOURCES/NGS_GRAVD_Block_CN03_BETA1/NGS_GRAVD_Block_CN03_Gravity_Data_BETA1.txt');
NGS_Block_place = NGS_Block(:,1);
NGS_Block_time = NGS_Block(:,2);
NGS_Block_latitude = NGS_Block(:,3);
NGS_Block_longitude = NGS_Block(:,4);
NGS_Block_elipsoidal_height = NGS_Block(:,5);
NGS_Block_gravity_filtered = NGS_Block(:,6);

# 2

In [2]:
%2 
%N_ggm = open('/Users/sunilbhavsar/Documents/CE678_PROJECT/Q2/RESOURCES/AIUB-CHAMP01S_a50de3dfc2ab0cdb9a2ae28b113f3398304202075281a813ba9cc27562a667af.gdf')
%a = open('/Users/sunilbhavsar/Documents/CE678_PROJECT/AIUB-CHAMP01S_a50de3dfc2ab0cdb9a2ae28b113f3398304202075281a813ba9cc27562a667af.gdf')
fid = fopen('Q2/Downloaded_data/GGM01C.gdf'); % file identifier

h = true; % while loop condition

n = 1; % headerlines counter

while true

    str = fgets(fid); % reading the file line by line
    
    if length(str) >= 11
        indx = strcmp(str(1:11), 'end_of_head'); % checking for 'end_of_head' string
    else
        indx = false;
    end
    
    if indx == 0
    
    n = n+1; % keep counting
    
    else
    
    h = false; % break loop
    break;
    
    end

end
str = fgets(fid);
fclose(fid);
GGM01C = textread('Q2/Downloaded_data/GGM01C.gdf','', 'headerlines', n);

In [3]:
GGM01C_longitudes = GGM01C(:,1)-360;
GGM01C_latitudes = GGM01C(:,2);
GGM01C_geoid = GGM01C(:,3);

In [4]:
%adding 
n_ggm = griddata(GGM01C_latitudes,GGM01C_longitudes,GGM01C_geoid,NGS_Block_latitude,NGS_Block_longitude,"nearest");
orthometric_height = n_ggm + NGS_Block_elipsoidal_height;

## 3

In [5]:
%3
a = 6378137.0;
b = 6356752.314140;
gamma_a = 9.7803267714;
gamma_b = 9.8321863685;
omega = 7292115.0*10^11;
GM = 3986004.418*10^8 ;
m = ((omega^2)*(a^2)*b)/GM;
f = (a-b)/a;
gamma = (a*gamma_a*(cosd(NGS_Block_latitude).^2) + b*gamma_b*(sind(NGS_Block_latitude).^2))./((a*(cosd(NGS_Block_latitude).^2)+b*(sind(NGS_Block_latitude).^2)).^0.5);
%calculating g_fac (free air correction)
term_1 = 0.3086*orthometric_height;
term2 = (3/a^2)*gamma.*(orthometric_height.^2);
g_fac = term_1 + term2;
free_air_anomily = NGS_Block_gravity_filtered + g_fac- gamma;

## 4

In [6]:
%4
fid = fopen('Q4/GGM01C_Anomily.gdf'); % file identifier

h = true; % while loop condition

n = 1; % headerlines counter

while true

    str = fgets(fid); % reading the file line by line
    
    if length(str) >= 11
        indx = strcmp(str(1:11), 'end_of_head'); % checking for 'end_of_head' string
    else
        indx = false;
    end
    
    if indx == 0
    
    n = n+1; % keep counting
    
    else
    
    h = false; % break loop
    break;
    
    end

end
str = fgets(fid);
fclose(fid);
GGM01C_anomily = textread('Q4/GGM01C_Anomily.gdf','', 'headerlines', n);
GGM01C_anomily_longitude = GGM01C_anomily(:,1)-360;
GGM01C_anomily_latitude = GGM01C_anomily(:,2);
GGM01C_anomily_gravity_anomily = GGM01C_anomily(:,4);

In [7]:
del_g_ggm = griddata(GGM01C_anomily_latitude,GGM01C_anomily_longitude,GGM01C_anomily_gravity_anomily,NGS_Block_latitude,NGS_Block_longitude);
del_g_s_and_mw = free_air_anomily-del_g_ggm;

## 5

In [8]:
%5
g_atm = 0.871 - 1.0298*(10^(-4))*orthometric_height + 5.3105*(10^-9)*(orthometric_height.^2)  - 2.1642*(10^-13)*(orthometric_height.^3)+ 9.5246*(10^-18)*(orthometric_height.^4) - 2.2411*(10^-22)*(orthometric_height.^5);
g_atm_and_s_mw = del_g_s_and_mw - g_atm;

## 6

In [14]:
%6
lat = min(NGS_Block_latitude):0.01:max(NGS_Block_latitude);
long = min(NGS_Block_longitude):0.01:max(NGS_Block_longitude);
size(lat)
[Latitude_grid,Longitude_grid]=meshgrid(lat,long);
g_atm_and_s_mw_grid = griddata(NGS_Block_latitude,NGS_Block_longitude,g_atm_and_s_mw,Latitude_grid,Longitude_grid,"nearest");

ans =

     1   293

